In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)

os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.metrics import  roc_auc_score

import pickle 
import logging


# from config.config import SQLQuery
# querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/plaid_credit
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code


In [2]:
from preprocess import Convert,MissingValues,Outlier,FeatureSelection
from feature_transformation import Scaler,Transform,Selection
from model_building import split_test_train, feature_encoding, classification_models
from model_evaluations import model_metrics, feature_importance, probability_bins, cross_validation
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

from statsmodels.stats.outliers_influence import variance_inflation_factor

# object initiation 
tf = Transform()
sel = Selection()
ft = FeatureSelection()
cv = Convert()
mv = MissingValues()
ot = Outlier()

# set seed
seed = 37

In [3]:
df_raw = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/testing_features.pkl')
df_raw.shape

(5, 10)

In [4]:
feat_list = ['credit_size_3M',
 'pos_size_1M',
 'ratio_amt_credits_less_100_credits_grt_500_3M',
 'median_credit_amt_1M',
 'ratio_pos_size_1M_6M',
 'median_pos_amt_3M',
 'count_credits_grt_500_3M',
 'loans_size_6M',
 'ach_count_6M']

In [7]:
keep_cols = ['business_id']

In [8]:
df_raw = df_raw[keep_cols + feat_list]
df_raw.shape

(5, 10)

In [9]:
df_raw.sample(5)

,business_id,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M
2,3c4f32e9-c92a-4fd5-9303-9365bf5f392e,65.053158,6.085,NaN,12.170,0.291334,29.83,NaN,NaN,NaN
1,3369b5d7-92a0-4d8a-813b-11f34abd34d6,2474.628333,NaN,0.000356,270.000,NaN,4791.28,12.0,NaN,2.0
0,07bb4be8-674f-40fb-acf7-116ca4ed257c,830.732500,NaN,0.008015,55.345,NaN,NaN,1.0,NaN,NaN
3,8105af48-e73f-4f4b-9913-6cb3153a7556,119.084595,NaN,0.013856,1.130,NaN,NaN,4.0,NaN,NaN
4,c95b45e5-9df5-496b-a6d2-a890544f58e7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_raw.describe()

,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M
count,4.000000,1.000,3.000000,4.000000,1.000000,2.000000,3.000000,0.0,1.0
mean,872.374646,6.085,0.007409,84.661250,0.291334,2410.555000,5.666667,NaN,2.0
std,1123.708864,NaN,0.006770,125.754186,NaN,3366.853583,5.686241,NaN,NaN
min,65.053158,6.085,0.000356,1.130000,0.291334,29.830000,1.000000,NaN,2.0
25%,105.576735,6.085,0.004186,9.410000,0.291334,1220.192500,2.500000,NaN,2.0
50%,474.908547,6.085,0.008015,33.757500,0.291334,2410.555000,4.000000,NaN,2.0
75%,1241.706458,6.085,0.010936,109.008750,0.291334,3600.917500,8.000000,NaN,2.0
max,2474.628333,6.085,0.013856,270.000000,0.291334,4791.280000,12.000000,NaN,2.0


In [11]:
median_dict = {'credit_size_3M' : 774.588458,
                'pos_size_1M' : 281.750000,
                'ratio_amt_credits_less_100_credits_grt_500_3M' : 9.341087e-03,
                'median_credit_amt_1M' : 369.600000,
                'ratio_pos_size_1M_6M' : 0.692535,
                'median_pos_amt_3M' : 300,
                'count_credits_grt_500_3M' : 10,
                'loans_size_6M' : 783.3,
                'ach_count_6M' : 5}

In [12]:
df_raw[df_raw['txn_flag'] == 0] = df_raw[df_raw['txn_flag'] == 0].fillna(median_dict)

KeyError: 'txn_flag'

In [13]:
df_raw = df_raw.fillna(0)
df_raw.isnull().sum().sum()

0

In [ ]:
df_raw.drop('txn_flag',axis=1,inplace=True)

In [ ]:
df_raw['target'] = df_raw['target'].astype('int')

In [ ]:
feat_list

['credit_size_3M',
 'pos_size_1M',
 'ratio_amt_credits_less_100_credits_grt_500_3M',
 'median_credit_amt_1M',
 'ratio_pos_size_1M_6M',
 'median_pos_amt_3M',
 'count_credits_grt_500_3M',
 'loans_size_6M',
 'ach_count_6M']

In [14]:
transformed_vars = df_raw.copy()

In [15]:
# 1
# transform
col         = 'credit_size_3M'
conditions  = [ transformed_vars[col] < 178.8,
                (transformed_vars[col] >= 178.9 ) & (transformed_vars[col] < 392.56),
                (transformed_vars[col] >= 392.56 ) & (transformed_vars[col] < 2274.8),
                transformed_vars[col] >= 2274.8 ]

choices     = [-1.056997,-0.484478,0.184337,1.294378]
    
transformed_vars["credit_size_3M"] = np.select(conditions, choices, default=np.nan)


# 2
# transform
col         = 'pos_size_1M'
conditions  = [ transformed_vars[col] < 235.11, 
                (transformed_vars[col] >= 235.11 ) & (transformed_vars[col] < 979.44), 
                transformed_vars[col] >= 979.44 ]

choices     = [-0.152312,0.159398,1.699843]
    
transformed_vars["pos_size_1M"] = np.select(conditions, choices, default=np.nan)

# 3
# transform
col         = 'ratio_amt_credits_less_100_credits_grt_500_3M'
conditions  = [ transformed_vars[col] <= 0,
                transformed_vars[col] > 0 ]

choices     = [-0.112536,0.045514]
    
transformed_vars["ratio_amt_credits_less_100_credits_grt_500_3M"] = np.select(conditions, choices, default=np.nan)

# 4
# transform
col         = 'median_credit_amt_1M'
conditions  = [ transformed_vars[col] < 102,
                (transformed_vars[col] >= 102 ) & (transformed_vars[col] < 475.2),
                transformed_vars[col] >= 475.2 ]

choices     = [-0.444738,-0.119068,0.550264]
    
transformed_vars["median_credit_amt_1M"] = np.select(conditions, choices, default=np.nan)


# 5
# transform
col         = 'ratio_pos_size_1M_6M'
conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 1.21),
                transformed_vars[col] >= 1.21 ]

choices     = [-0.164722,0.141314,1.140227]
    
transformed_vars["ratio_pos_size_1M_6M"] = np.select(conditions, choices, default=np.nan)

# 6
# transform
col         = 'median_pos_amt_3M'
conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 486.25),
                transformed_vars[col] >= 486.25 ]

choices     = [-0.25334,-0.127284,0.968955]
    
transformed_vars["median_pos_amt_3M"] = np.select(conditions, choices, default=np.nan)

# 7
# transform
col         = 'count_credits_grt_500_3M'
conditions  = [ transformed_vars[col] < 1.5,
                (transformed_vars[col] >= 1.5 ) & (transformed_vars[col] < 15.5),
                transformed_vars[col] >= 15.5 ]

choices     = [-0.450862,-0.000945,0.284561]
    
transformed_vars["count_credits_grt_500_3M"] = np.select(conditions, choices, default=np.nan)

# 8
# transform
col         = 'loans_size_6M'
conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 3317),
                transformed_vars[col] >= 3317 ]

choices     = [0.08126,-0.605012,0.814805]
    
transformed_vars["loans_size_6M"] = np.select(conditions, choices, default=np.nan)


# 9
# transform
col         = 'ach_count_6M'
conditions  = [ transformed_vars[col] < 6.5,
                transformed_vars[col] >= 6.5 ]

choices     = [-0.022101,0.149246]
    
transformed_vars["ach_count_6M"] = np.select(conditions, choices, default=np.nan)


In [16]:
transformed_vars.head()

,business_id,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M
0,07bb4be8-674f-40fb-acf7-116ca4ed257c,0.184337,-0.152312,0.045514,-0.444738,-0.164722,-0.253340,-0.450862,0.08126,-0.022101
1,3369b5d7-92a0-4d8a-813b-11f34abd34d6,1.294378,-0.152312,0.045514,-0.119068,-0.164722,0.968955,-0.000945,0.08126,-0.022101
2,3c4f32e9-c92a-4fd5-9303-9365bf5f392e,-1.056997,-0.152312,-0.112536,-0.444738,0.141314,-0.127284,-0.450862,0.08126,-0.022101
3,8105af48-e73f-4f4b-9913-6cb3153a7556,-1.056997,-0.152312,0.045514,-0.444738,-0.164722,-0.253340,-0.000945,0.08126,-0.022101
4,c95b45e5-9df5-496b-a6d2-a890544f58e7,-1.056997,-0.152312,-0.112536,-0.444738,-0.164722,-0.253340,-0.450862,0.08126,-0.022101


In [17]:
logreg_model = pickle.load(open('/Users/shashankgupta/Documents/code/git_project/plaid_credit/models/final_model.pkl', 'rb'))

In [18]:
# Scoring on full-data

scored_df = transformed_vars.copy()
scored_df['pred_proba'] = logreg_model.predict_proba(scored_df[feat_list])[:,1]

scored_df['odds'] = scored_df['pred_proba'] / (1-scored_df['pred_proba'])
scored_df['log_odds'] = np.log(scored_df['odds'])
scored_df['Ext_IN1_score'] = 601.7 - (86.6 * scored_df['log_odds'])
scored_df.Ext_IN1_score.describe()

count      5.000000
mean     576.880082
std       74.568054
min      494.874938
25%      538.510618
50%      549.255823
75%      616.750438
max      685.008591
Name: Ext_IN1_score, dtype: float64

In [22]:
scored_df[['business_id','Ext_IN1_score']].to_clipboard()

In [ ]:
def bin_logic(x):
    if x < 0.261540:
        return 5
    if x < 0.455167:
        return 4
    if x < 0.530817:
        return 3
    if x < 0.628923:
        return 2
    return 1

In [ ]:
scored_df['Ext_IN1_bin'] = scored_df['pred_proba'].apply(lambda x: bin_logic(x))

In [21]:
scored_df.to_clipboard()

In [ ]:
scored_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 646 entries, 0 to 645
Data columns (total 18 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   business_id                                    646 non-null    object 
 1   lending_business_id                            646 non-null    object 
 2   decision_date                                  646 non-null    object 
 3   target                                         646 non-null    int64  
 4   credit_size_3M                                 646 non-null    float64
 5   pos_size_1M                                    646 non-null    float64
 6   ratio_amt_credits_less_100_credits_grt_500_3M  646 non-null    float64
 7   median_credit_amt_1M                           646 non-null    float64
 8   ratio_pos_size_1M_6M                           646 non-null    float64
 9   median_pos_amt_3M                              646 non-null

In [ ]:
t = scored_df.groupby('Ext_IN1_bin',as_index=False)['target'].agg(['sum','count']).rename(columns={'sum':'count_of_bads','count':'total_accs'})
t['bad_rate'] = np.round(t['count_of_bads']/t['total_accs'],2)
t

,Ext_IN1_bin,count_of_bads,total_accs,bad_rate
0,1,55,139,0.40
1,2,28,116,0.24
2,3,29,149,0.19
3,4,15,124,0.12
4,5,6,118,0.05


In [ ]:
t1 = scored_df.groupby('Ext_IN1_bin', as_index=False)['Ext_IN1_score'].min().rename(columns={'Ext_IN1_score':'min_score'})
t2 = scored_df.groupby('Ext_IN1_bin', as_index=False)['Ext_IN1_score'].max().rename(columns={'Ext_IN1_score':'max_score'})

t3 = scored_df.groupby('Ext_IN1_bin', as_index=False)['pred_proba'].min().rename(columns={'pred_proba':'min_prob'})
t4 = scored_df.groupby('Ext_IN1_bin', as_index=False)['pred_proba'].max().rename(columns={'pred_proba':'max_prob'})

res = t1.merge(t2, on='Ext_IN1_bin')
res = res.merge(t3, on='Ext_IN1_bin')
res = res.merge(t4, on='Ext_IN1_bin')
res.to_clipboard()

In [ ]:
scored_df.groupby('Ext_IN1_bin', as_index=False)['Ext_IN1_score'].max()

,Ext_IN1_bin,Ext_IN1_score
0,1,469.821422
1,2,557.045827
2,3,591.011509
3,4,619.916659
4,5,691.589243


In [ ]:
logreg_model.feature_names_in_

array(['credit_size_3M', 'pos_size_1M',
       'ratio_amt_credits_less_100_credits_grt_500_3M',
       'median_credit_amt_1M', 'ratio_pos_size_1M_6M',
       'median_pos_amt_3M', 'count_credits_grt_500_3M', 'loans_size_6M',
       'ach_count_6M'], dtype=object)

In [ ]:
logreg_model.coef_

array([[-0.81997815, -0.85685935,  0.72385091, -0.34517684, -0.35318817,
        -0.1268506 ,  0.86565206, -0.42155525,  0.18572408]])

In [ ]:
logreg_model.intercept_

array([-0.00127315])

In [ ]:
# scored_df.to_csv('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/scored_dataset.csv')

In [ ]:
# scored_df.to_pickle('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/scored_dataset.pkl')